# PBMC 3K Dataset Analysis

This notebook demonstrates a complete scRNA-seq analysis using Scanpy (Python) on the PBMC 3K dataset from 10x Genomics.

In [ ]:
# Import necessary libraries
import scanpy as sc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
sc.settings.verbosity = 3
sc.settings.set_figure_params(dpi=80)

## Data Loading

First, download the PBMC 3K dataset from 10x Genomics if you haven't already:
```
# In terminal (optional):
# mkdir -p ../data
# wget https://cf.10xgenomics.com/samples/cell/pbmc3k/pbmc3k_filtered_gene_bc_matrices.tar.gz -O ../data/pbmc3k_filtered_gene_bc_matrices.tar.gz
# tar -xzf ../data/pbmc3k_filtered_gene_bc_matrices.tar.gz -C ../data/
```

In [ ]:
# Load the data
# If using 10x HDF5 format
# adata = sc.read_10x_h5('../data/pbmc_3k_filtered_feature_bc_matrix.h5')

# If using 10x MTX format
adata = sc.read_10x_mtx(
    '../data/filtered_gene_bc_matrices/hg19/',  # Adjust path as necessary
    var_names='gene_symbols',
    cache=True
)

# Basic information
print(f"Shape: {adata.shape}")
adata

## Quality Control

In [ ]:
# Calculate QC metrics
adata.var['mt'] = adata.var_names.str.startswith('MT-')
sc.pp.calculate_qc_metrics(
    adata, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True
)

# Plot QC metrics
fig, axs = plt.subplots(1, 3, figsize=(15, 4))
sc.pl.violin(adata, 'n_genes_by_counts', jitter=0.4, ax=axs[0])
sc.pl.violin(adata, 'total_counts', jitter=0.4, ax=axs[1])
sc.pl.violin(adata, 'pct_counts_mt', jitter=0.4, ax=axs[2])
plt.tight_layout()

In [ ]:
# Filter cells
sc.pp.filter_cells(adata, min_genes=200)
sc.pp.filter_cells(adata, max_genes=2500)  # Remove potential doublets
adata = adata[adata.obs.pct_counts_mt < 5, :]

# Filter genes
sc.pp.filter_genes(adata, min_cells=3)

print(f"After filtering: {adata.shape}")

## Normalization and Scaling

In [ ]:
# Normalize to depth of 10,000 reads per cell
sc.pp.normalize_total(adata, target_sum=1e4)

# Log-transform
sc.pp.log1p(adata)

# Find highly variable genes
sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5)
print(f"Number of highly variable genes: {sum(adata.var.highly_variable)}")

# Plot variable genes
sc.pl.highly_variable_genes(adata)

In [ ]:
# Use only highly variable genes for downstream analysis
adata = adata[:, adata.var.highly_variable]

# Scale data
sc.pp.scale(adata, max_value=10)

## Dimensionality Reduction and Clustering

In [ ]:
# Run PCA
sc.tl.pca(adata, svd_solver='arpack')

# Plot PCA
sc.pl.pca(adata, color='n_genes_by_counts')
sc.pl.pca_variance_ratio(adata, n_pcs=50)

In [ ]:
# Compute the neighborhood graph
sc.pp.neighbors(adata, n_neighbors=10, n_pcs=30)

# UMAP embedding
sc.tl.umap(adata)

# Find clusters using Leiden algorithm
sc.tl.leiden(adata, resolution=0.5)

# Plot UMAP
sc.pl.umap(adata, color=['leiden', 'n_genes_by_counts', 'pct_counts_mt'])

## Identify Marker Genes and Annotate Clusters

In [ ]:
# Find marker genes for each cluster
sc.tl.rank_genes_groups(adata, 'leiden', method='t-test')
sc.pl.rank_genes_groups(adata, n_genes=25, sharey=False)

In [ ]:
# View marker genes for specific clusters
pd.DataFrame(adata.uns['rank_genes_groups']['names']).head(10)

In [ ]:
# Define a list of known marker genes for PBMC cell types
marker_genes = {
    'T-cell': ['CD3D', 'CD3E', 'IL7R'],
    'CD4 T-cell': ['CD3D', 'IL7R', 'CD4'],
    'CD8 T-cell': ['CD3D', 'CD8A', 'CD8B'],
    'B-cell': ['MS4A1', 'CD79A', 'CD79B'],
    'NK cell': ['GNLY', 'NKG7', 'KLRD1'],
    'Monocyte': ['LYZ', 'CST3', 'CD14', 'FCGR3A'],
    'Conventional DC': ['FCER1A', 'CST3'],
    'Plasmacytoid DC': ['LILRA4', 'CLEC4C', 'IL3RA'],
    'Platelet': ['PPBP']
}

# Plot expression of markers to help with annotation
sc.pl.umap(adata, color=['leiden'], legend_loc='on data')

# Plot some selected markers
sc.pl.umap(adata, color=['CD3D', 'CD3E', 'MS4A1', 'CD79A', 'LYZ', 'GNLY'])

In [ ]:
# Based on marker gene expression, annotate clusters
# This is a manual process guided by domain knowledge
cell_type_map = {
    '0': 'CD4 T-cell',
    '1': 'CD14+ Monocyte',
    '2': 'B-cell',
    '3': 'CD8 T-cell',
    '4': 'NK cell',
    '5': 'FCGR3A+ Monocyte',
    '6': 'Dendritic cell',
    '7': 'Platelet'
    # Add more as needed
}

# Add cell type annotations to the object
adata.obs['cell_type'] = adata.obs['leiden'].map(cell_type_map).astype('category')

# Plot with cell type annotations
sc.pl.umap(adata, color='cell_type', legend_loc='on data')

## Save Results

In [ ]:
# Save to file
adata.write('../data/pbmc3k_processed.h5ad')

# Export key results to CSV for sharing
adata.obs[['cell_type', 'leiden', 'n_genes_by_counts', 'pct_counts_mt']].to_csv('../data/pbmc3k_metadata.csv')